## REPASO DE CLASIFICACIÓN
## Flujo de trabajo con pipelines

Generaremos un flujo de trabajo para abordar un problema de clasificación a partir del dataset de cancer de mama de Wisconsin, ya conocido. Aquí veremos Regresión Logística y SVM, luego pueden adicionar NB Gausiano y KNN al análisis/repaso.

In [ ]:
# Cargamos el dataset

import pandas as pd

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases'
                 '/breast-cancer-wisconsin/wdbc.data', header=None)

print(df.shape)
df.head()

In [ ]:
# Convertimos la variable target a numérica

from sklearn.preprocessing import LabelEncoder

X = df.loc[:, 2:].values
y = df.loc[:, 1].values

le = LabelEncoder()
y = le.fit_transform(y)

le.classes_

In [ ]:
# Generamos el split de nuestro dataset

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.30, stratify=y, random_state=1)

## Regresión logística

#### Implementamos un pipeline

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

# PIPELINE: encapsulamos los procesos de estandarización de features, reducción de dimensionalidad y modelado

pipe_lr = make_pipeline(StandardScaler(),
                        PCA(n_components=2),
                        LogisticRegression(random_state=1))

# Fiteamos el modelo
pipe_lr.fit(X_train, y_train)

# Implementamos el modelo para luego evaluarlo
y_pred = pipe_lr.predict(X_test)

print('Test Accuracy: %.3f' % pipe_lr.score(X_test, y_test))

En primera instancia, el modelo da muy bien... Aunque aún no hemos incluido cross validation...

#### Validación del modelo con K-Fold

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
    
# Generamos el mecanismo de partición del train sest

kfold = StratifiedKFold(n_splits=5, random_state=1, shuffle=True).split(X_train, y_train)

# Iteramos sobre cada partición y registramos scores

scores = []
for k, (train, test) in enumerate(kfold):
    
    pipe_lr.fit(X_train[train], y_train[train])
    score = pipe_lr.score(X_train[test], y_train[test])
    scores.append(score)
    
    print('Fold: %2d, Class dist.: %s, Acc: %.3f' % \
          (k+1,
           # np.bincount cuenta la ocurrencia de cada clase en los splits. Notar que cada split está balanceado.
           np.bincount(y_train[train]), 
           score))
    
print('\nCV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

In [ ]:
from sklearn.model_selection import cross_val_score

# Generamos CV equivalente o alternativo

scores = cross_val_score(estimator=pipe_lr,
                         X=X_train,
                         y=y_train,
                         cv=5,
                         n_jobs=1)

print('CV accuracy: %s' % scores)
print('Resumen CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

In [ ]:
from sklearn.metrics import confusion_matrix

# Imprimimos la matriz de confusión

confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(confmat)

## Curvas de aprendizaje

`learning_curve` itera las particiones del dataset en train y test hasta encontrar la cantidad óptima de samples, es decir, el punto donde se estabiliza el score sobre el test por más que incorporemos un mayor número de samples al train set. De esta forma, podemos eliminar el over-fitting garantizando el mínimo sesgo. Documentación [aquí](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.learning_curve.html).

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve

# Modificamos nuestro PIPELINE original
# Quitamos PCA e incluimos penalización L2
pipe_lr = make_pipeline(StandardScaler(),
                        LogisticRegression(penalty='l2', random_state=1))

# Implementamos una learning_curve
train_sizes, train_scores, test_scores =\
                learning_curve(estimator=pipe_lr,
                               X=X_train,
                               y=y_train,
                               train_sizes=np.linspace(0.1, 1.0, 10),
                               cv=10,
                               n_jobs=1)

# Calculamos una medida resumen de las estimadores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)



In [ ]:
# Graficamos
plt.plot(train_sizes, train_mean,
         color='blue', marker='o',
         markersize=5, label='training accuracy')

plt.fill_between(train_sizes,
                 train_mean + train_std,
                 train_mean - train_std,
                 alpha=0.15, color='blue')

plt.plot(train_sizes, test_mean,
         color='green', linestyle='--',
         marker='s', markersize=5,
         label='validation accuracy')

plt.fill_between(train_sizes,
                 test_mean + test_std,
                 test_mean - test_std,
                 alpha=0.15, color='green')

plt.grid()
plt.xlabel('Number of training samples')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim([0.8, 1.03])
plt.tight_layout()

plt.show()

## Curva de validación

`validation_curve` permite evaluar la performance de nuestro modelo (el pipeline en este caso) con distintos valores del hiperparámetro asociado, es una alternativa para lidiar con el over-fitting y evitar el under-fitting o una mala especificación del modelo. Documentación [aquí](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.validation_curve.html).

In [ ]:
from sklearn.model_selection import validation_curve

# Generamos valores para el hiperparámetro C de la regresión logística
param_range = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]

# Implementamos una curva de validación
train_scores, test_scores = validation_curve(
                estimator=pipe_lr, 
                X=X_train, 
                y=y_train, 
                param_name='logisticregression__C', 
                param_range=param_range,
                cv=10)

# Calculamos una medida resumen de las estimadores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)


In [ ]:

#Graficamos
plt.plot(param_range, train_mean, 
         color='blue', marker='o', 
         markersize=5, label='training accuracy')

plt.fill_between(param_range, train_mean + train_std,
                 train_mean - train_std, alpha=0.15,
                 color='blue')

plt.plot(param_range, test_mean, 
         color='green', linestyle='--', 
         marker='s', markersize=5, 
         label='validation accuracy')

plt.fill_between(param_range, 
                 test_mean + test_std,
                 test_mean - test_std, 
                 alpha=0.15, color='green')

plt.grid()
plt.xscale('log')
plt.legend(loc='lower right')
plt.xlabel('Parameter C')
plt.ylabel('Accuracy')
plt.ylim([0.8, 1.0])
plt.tight_layout()
# plt.savefig('images/06_06.png', dpi=300)
plt.show()

## SVM

## Tuneo de hiperparámetros con GridSearch

Como hemos visto, GridSearch es una alternativa para determinar cuál es el mejor hiperparámetro (o combinaciones de hiperparámetros y/o kernels).

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Generamos el pipeline para SVM
pipe_svc = make_pipeline(StandardScaler(),
                         SVC(random_state=1))

# Generamos valores para los hiperparámetros C y gamma
param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

# Generamos el grid de parámetros y kernel correspondiente
param_grid = [{'svc__C': param_range, 
               'svc__kernel': ['linear']},
              {'svc__C': param_range, 
               'svc__gamma': param_range, 
               'svc__kernel': ['rbf']}]

# Implementamos GridSearch
gs = GridSearchCV(estimator=pipe_svc, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  cv=10,
                  n_jobs=-1)

gs = gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

In [ ]:
# Tomamos los mejores estimadores
svm = gs.best_estimator_
svm.fit(X_train, y_train)
print('Test accuracy: %.3f' % svm.score(X_test, y_test))

In [ ]:
# Imprimimos la matriz de confusión
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(confmat)

#### Secuencia de validación

In [ ]:
gs = GridSearchCV(estimator=pipe_svc,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=2)

scores = cross_val_score(gs, X_train, y_train, 
                         scoring='accuracy', cv=5)

print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),
                                      np.std(scores)))

## Medidas de evaluación de los modelos

#### Matriz de confusión

Evaluamos la matriz de confusión de SVM.

In [ ]:
from sklearn.metrics import confusion_matrix

pipe_svc.fit(X_train, y_train)
y_pred = pipe_svc.predict(X_test)
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(confmat)

In [ ]:
fig, ax = plt.subplots(figsize=(2.5, 2.5))
ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)
for i in range(confmat.shape[0]):
    for j in range(confmat.shape[1]):
        ax.text(x=j, y=i, s=confmat[i, j], va='center', ha='center')

plt.xlabel('Predicted label')
plt.ylabel('True label')

plt.tight_layout()
#plt.savefig('images/06_09.png', dpi=300)
plt.show()

¿Qué nos está diciendo el modelo?

## Métricas de evaluación

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

print('Precision: %.3f' % precision_score(y_true=y_test, y_pred=y_pred))
print('Recall: %.3f' % recall_score(y_true=y_test, y_pred=y_pred))
print('F1: %.3f' % f1_score(y_true=y_test, y_pred=y_pred))

Una situación común es que querramos que el "best estimator" de cross validation sea una métrica en particular, por ejemplo, el f1 score.

In [ ]:
from sklearn.metrics import make_scorer

# Implementamos make_scorer para evaluar con GridSearch
scorer = make_scorer(f1_score, pos_label=0)

c_gamma_range = [0.01, 0.1, 1.0, 10.0]

param_grid = [{'svc__C': c_gamma_range,
               'svc__kernel': ['linear']},
              {'svc__C': c_gamma_range,
               'svc__gamma': c_gamma_range,
               'svc__kernel': ['rbf']}]

# Incluimos el scorer en el GridSearch
gs = GridSearchCV(estimator=pipe_svc,
                  param_grid=param_grid,
                  scoring=scorer,
                  cv=10,
                  n_jobs=-1)

gs = gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

#### ROC curve y AUC

In [ ]:
from sklearn.metrics import roc_curve, auc
from scipy import interp

# Cambiamos nuestro pipeline de LR
pipe_lr = make_pipeline(StandardScaler(),
                        PCA(n_components=2),
                        LogisticRegression(penalty='l2', 
                                           random_state=1, 
                                           C=100.0))

X_train2 = X_train[:, [4, 14]]

# Instrumentamos CV
cv = list(StratifiedKFold(n_splits=3, 
                          random_state=1).split(X_train, y_train))

# Graficamos
fig = plt.figure(figsize=(7, 5))

mean_tpr = 0.0
mean_fpr = np.linspace(0, 1, 100)
all_tpr = []

# Iteramos el pipeline sobre CV, calculando probabilidades de la predicción y la curva ROC
for i, (train, test) in enumerate(cv):
    probas = pipe_lr.fit(X_train2[train],
                         y_train[train]).predict_proba(X_train2[test])

    fpr, tpr, thresholds = roc_curve(y_train[test],
                                     probas[:, 1],
                                     pos_label=1)
    mean_tpr += interp(mean_fpr, fpr, tpr)
    mean_tpr[0] = 0.0
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr,
             tpr,
             label='ROC fold %d (area = %0.2f)'
                   % (i+1, roc_auc))

plt.plot([0, 1],
         [0, 1],
         linestyle='--',
         color=(0.6, 0.6, 0.6),
         label='random guessing')

mean_tpr /= len(cv)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, 'k--',
         label='mean ROC (area = %0.2f)' % mean_auc, lw=2)
plt.plot([0, 0, 1],
         [0, 1, 1],
         linestyle=':',
         color='black',
         label='perfect performance')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.legend(loc="lower right")

plt.tight_layout()
# plt.savefig('images/06_10.png', dpi=300)
plt.show()